#INSTALA LIBRERIAS

Se descargan librerias de GitHub necesarias para ejecutar el algoritmo, se puede comentariar si se ejecutan varios experimentos, solo correr 1 vez

In [3]:
# DESCARGA DE LIBRERIAS
#!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git@2070f85cdd4d2cde4c00da98ff01ff448b1e593b --use-deprecated=legacy-resolver
#!pip install git+https://github.com/quantopian/pyfolio.git --use-deprecated=legacy-resolver
#!pip install yfinance --use-deprecated=legacy-resolver

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git (to revision 2070f85cdd4d2cde4c00da98ff01ff448b1e593b) to /tmp/pip-req-build-5dwkec8b
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-5dwkec8b
  Running command git rev-parse -q --verify 'sha^2070f85cdd4d2cde4c00da98ff01ff448b1e593b'
  Running command git fetch -q https://github.com/AI4Finance-LLC/FinRL-Library.git 2070f85cdd4d2cde4c00da98ff01ff448b1e593b
  Running command git checkout -q 2070f85cdd4d2cde4c00da98ff01ff448b1e593b
     |████████████████████████████████| 91 kB 9.1 MB/s 
     |████████████████████████████████| 176 kB 48.5 MB/s 
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 1.6 MB 45.0 MB/s 
     |████████████████████████████████| 2.4 MB 38.9 MB/s 
     |████████████████████████████████| 6.4 MB 29.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 52 kB 1.5 M

  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-req-build-_vlfg8vo
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-req-build-_vlfg8vo
     |████████████████████████████████| 245 kB 20.7 MB/s 
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=75774 sha256=ef860da05b3f6a2ba449c08b91f9f22d67fd0436f5e11ebffded747ebdfcbe59
  Stored in directory: /tmp/pip-ephem-wheel-cache-70eve64p/wheels/ef/09/e5/2c1bf37c050d22557c080deb1be986d06424627c04aeca19b9
Successfully built pyfolio
  Attempting uninstall: pyfolio
    Found existing installation: pyfolio 0.9.2
    Uninstalling pyfolio-0.9.2:
      Successfully uninstalled pyfolio-0.9.2
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.28
    Uninstalling prompt-toolkit-3.0.28:
      Successfully uninstalled prompt-toolkit-3.0.28
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting pack

#Descarga de librerias

In [4]:
# DESCARGA DE LIBRERIAS 2
import itertools
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime
import os
import math

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_portfolio import StockPortfolioEnv

from finrl.model.models import DRLAgent

from scipy import stats

from gym.utils import seeding
import gym
from gym import spaces
from stable_baselines3.common.vec_env import DummyVecEnv

import yfinance as yf
from pandas_datareader import data as pdr

import pyfolio
from copy import deepcopy

  # para usar el sr como reward funct
  # Se define el nombre de la carpeta o directorio a crear
directorio = "./data"
try:
  os.mkdir(directorio)
except OSError:
  print("La creación del directorio %s falló" % directorio)
else:
  print("Se ha creado el directorio: %s " % directorio)

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


Se ha creado el directorio: ./data 


# AGENTE

En esta sección se configuran los ajustes del agente:
 - PPO Policy
 - actor, es la red neuronal (Figura de actor, hasta Miu), es a partir del ELSE
        1. cambiar funcion de activación con el mismo numero de capas
        2. cambiar optimizador
        3. cambiar número de capas (no tienen q ser 64)
 - critic: Tiene 1 neurona de salida, puede agregarse otra capa de salida
 


In [5]:
# AGENT DE APRENDIZAJE POR REFUERZO
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
import time

################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")


################################## PPO Policy ##################################


class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self,
                 state_dim,
                 action_dim,
                 has_continuous_action_space,
                 action_std_init,
                 nn_arch,
                 kernel_size):
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_dim = action_dim
            #[0.6, 0.6, 0.6]
            # Calcular la varianza los precios de cierre historicos de cada activo
            self.action_var = torch.full((action_dim,), action_std_init * action_std_init).to(device)

        self.nn_arch = nn_arch

        # actor
        # 1. cambiar funcion de activación con el mismo nuomero de capas
        # 2. cambiar optimizador
        # 3. cambiar número de capas
        if has_continuous_action_space:
            if self.nn_arch == 'CNN-MLP':
              self.actor = nn.Sequential(
                              nn.Linear(state_dim, state_dim),
                              nn.AvgPool1d(kernel_size, stride=kernel_size),
                              #nn.Conv1d(1, 1, kernel_size, stride=kernel_size),
                              #2d, #imprimir diagrama, #downsampling, upsampling
                              nn.Linear(int(state_dim / kernel_size), 64),
                              nn.Tanh(),
                              nn.Linear(64, 64),
                              nn.Tanh(),
                              nn.Linear(64, action_dim),
                              nn.Tanh())
    ######################### AQUI SE MODIFICA #############################
            else:
              self.actor = nn.Sequential(
                              nn.Linear(state_dim, 64),
                              nn.Tanh(),
                              nn.Linear(64, 64),
                              nn.Tanh(),
                              nn.Linear(64, action_dim),
                              nn.Tanh())
                              
        # critic
        if self.nn_arch == 'CNN-MLP':
            self.critic = nn.Sequential(
                            nn.Linear(state_dim, state_dim),
                            nn.AvgPool1d(kernel_size, stride=kernel_size),
                            #nn.Conv1d(1, 1, kernel_size, stride=kernel_size),
                            nn.Linear(int(state_dim / kernel_size), 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, 1))
    ######################### AQUI SE MODIFICA #############################
        else:
          self.critic = nn.Sequential(
                          nn.Linear(state_dim, 64),
                          nn.Tanh(),
                          nn.Linear(64, 64),
                          nn.Tanh(),
                          nn.Linear(64, 1))
                          #nn.Tanh())
                
        
    def set_action_std(self, new_action_std):
        #return
        if self.has_continuous_action_space:
            self.action_var = torch.full((self.action_dim,), new_action_std * new_action_std).to(device)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def forward(self):
        raise NotImplementedError
    

    def act(self, state):
        if self.has_continuous_action_space:
            action_mean = self.actor(state) #LA SALIDA ES MIU
            cov_mat = torch.diag(self.action_var).unsqueeze(dim=0) #Calcula la matriz de covarianzas
            dist = MultivariateNormal(action_mean, cov_mat) #Genera la matriz de distribución normal con (M,N) o sea la funcion
        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action = dist.sample() # Genera numeros aleatorios y se los pasa a la funcion de distribucion que se deriva de las medias de las acciones y la matriz de covarianza (dist)
        action_logprob = dist.log_prob(action) #La accion es la flechita que sale de N en el diagrama
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state, action): #Es similar a actuar, pero evalua que tan buenos son los valores de los estados que se han pronosticado
        if self.nn_arch == 'CNN-MLP':
          state = state.unsqueeze(0)
        if self.has_continuous_action_space:
            action_mean = self.actor(state)
            
            action_var = self.action_var.expand_as(action_mean)
            cov_mat = torch.diag_embed(action_var).to(device)
            dist = MultivariateNormal(action_mean, cov_mat)
            
            # For Single Action Environments.
            if self.action_dim == 1:
                action = action.reshape(-1, self.action_dim)
        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self,
                 state_dim,
                 action_dim,
                 lr_actor,
                 lr_critic,
                 gamma,
                 K_epochs,
                 eps_clip,
                 has_continuous_action_space,
                 action_std_init=0.6,
                 nn_arch='MLP',
                 kernel_size=1):

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim,
                                  action_dim,
                                  has_continuous_action_space,
                                  action_std_init,
                                  nn_arch,
                                  kernel_size).to(device)
        print(self.policy)
        self.optimizer = torch.optim.RMSprop([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim,
                                      action_dim,
                                      has_continuous_action_space,
                                      action_std_init,
                                      nn_arch,
                                      kernel_size).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def set_action_std(self, new_action_std):
        
        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)
        
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def decay_action_std(self, action_std_decay_rate, min_action_std):
        print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        print("--------------------------------------------------------------------------------------------")


    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            state_unsqueeze = state.unsqueeze(0)
            action, action_logprob = self.policy_old.act(state_unsqueeze)
  
        if self.has_continuous_action_space:
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)

            return action.detach().cpu().numpy().flatten()
        else:
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)

            return action.item()


    def update(self):
      # Monte Carlo estimate of returns
      rewards = []
      discounted_reward = 0
      for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
          if is_terminal:
              discounted_reward = 0
          discounted_reward = reward + (self.gamma * discounted_reward)
          rewards.insert(0, discounted_reward)
          
      # Normalizing the rewards
      rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
      rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

      # convert list to tensor
      old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
      old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
      old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

      
      # Optimize policy for K epochs
      for _ in range(self.K_epochs):

        # Evaluating old actions and values
        logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

        # match state_values tensor dimensions with rewards tensor
        state_values = torch.squeeze(state_values)
        
        # Finding the ratio (pi_theta / pi_theta__old)
        ratios = torch.exp(logprobs - old_logprobs.detach())

        # Finding Surrogate Loss
        advantages = rewards - state_values.detach()   
        surr1 = ratios * advantages
        surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

        # final loss of clipped objective PPO
        loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
        
        # take gradient step
        self.optimizer.zero_grad()
        loss.mean().backward()
        self.optimizer.step()
      # Copy new weights into old policy
      self.policy_old.load_state_dict(self.policy.state_dict())

      # clear buffer
      self.buffer.clear()
            
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

Device set to : cpu


#PPO

In [6]:
class PortfolioAllocation(gym.Env):
  passmetadata = {'render.modes': ['human']}

  def __init__(self,
               df,
               stock_dim,
               initial_amount,
               tech_indicator_list,
               market_variables,
               reward_function,
               day=0,
               data_stddev=None,
               free_risk_return=None,
               data_corr=None,
               expected_values=None,
               trading_fee=0,
               nn_arch='MLP',
               window_width=1):
    self.df = df
    self.stock_dim = stock_dim
    self.initial_amount = initial_amount
    self.tech_indicator_list = tech_indicator_list
    self.day_start = day
    self.day = self.day_start
    self.market_variables = market_variables
    self.reward_function = reward_function
    self.window_width = window_width
    self.nn_arch = nn_arch

    self.setup()

    # Reward based on Sharpe Ratio (correlation)
    self.expected_values = expected_values
    self.data_stddev = data_stddev
    self.data_corr = data_corr
    self.free_risk_return = free_risk_return

    self.trading_fee = trading_fee

  def setup(self):
    # action_space normalization and shape is self.stock_dim
    self.action_space = spaces.Box(low=0, high=1,
                                   shape=(self.stock_dim,))
    # observation space
    shape = (self.stock_dim * (len(self.market_variables) + len(self.tech_indicator_list)) * self.window_width,)

    self.observation_space = spaces.Box(low=-0, high=np.inf,
                                        shape=shape)

    self.reset()

  def step(self, actions):
    self.terminal = self.day >= len(self.df.index.unique()) - 1

    if self.terminal:
      #df = pd.DataFrame(self.portfolio_return_memory)
      #df.columns = ['daily_return']
      return self.state, self.reward, self.terminal, {}
    else:
      # actions are the portfolio weight
      # normalize to sum of 1
      weights = self.softmax_normalization(actions) #-> [0.2, 0.7, 0.1] #Los pesos son el capital a destinar de cada uno de los activos, se calculan a paritr de aplicar una normalizacion de tipo softmax a lo que se considera actions
      # Acciones son la salida de la N

      last_day_memory = self.data

      # load next state
      self.day += 1
      self.state = self._get_state()
  
      prev_portfolio_value = self.portfolio_value
      # [0.2, 0.7, 0.1] * [80, 120] -> [0.2, 0.7, 0.1] * [80, 120, 1] 
      # 1 am = [1] / [120] = 0.0083 (1dll)
      # 2 [0.0083]* [150] = 1.25  (+ 0.25dll)
      # 3 am += ([1] * [1.25]) / [150] = 
      # A [0.5, 0.5] / [120, 80] = 0.0041, 0.0062
      # B [0.0041, 0.0062] * [130, 100] = [ 0.54 , 0.62] = 1.16
      # C [0.3, 0.7] * [1.16] = [0.384, 0.82] / [130, 100] = [0.00276259, 0.0082] = 
      self.portfolio_value = sum(self.actions_memory[-1] * self.data.close.values)
      self.actions_memory.append(weights*self.portfolio_value / self.data.close.values)
      portfolio_return = ((self.portfolio_value / prev_portfolio_value) - 1)

      # save into memory
      self.portfolio_return_memory.append(portfolio_return)
      self.date_memory.append(self.data.date.unique()[0])           
      self.asset_memory.append(self.portfolio_value)

      # reward

      if self.reward_function == 'pv':
        self.reward = portfolio_return
        #self.reward = portfolio_return
      elif self.reward_function == 'sr':
        portfolio_risk = self.portfolio_risk(weights, self.data_stddev, self.data_corr)
        self.reward = self.sharpeVE_ratio(weights, self.expected_values, portfolio_risk, self.free_risk_return)

      return self.state, self.reward, self.terminal, {}

  def _get_state(self):
    self.data = self.df.loc[self.day,:].sort_values(by=['tic'])

    tmp_data = [np.array(self.data[prop])
                for prop in self.market_variables + self.tech_indicator_list]

    for i in range(1, self.window_width):
      data = self.df.loc[self.day - i,:].sort_values(by=['tic'])
      tmp_data = tmp_data + [np.array(data[prop])
                             for prop in self.market_variables + self.tech_indicator_list]
    
    state = np.concatenate(tmp_data, axis=0)
    return state

  def _reset(self):
    self.state = self._get_state()
    self.terminal = False
    # Agregar + 1 en stock_dim 1/(self.stock_dim + 1)
    initial_weights = np.array([1/self.stock_dim] * self.stock_dim)
    # [.5, .5] * [120, 80] = [0.004, 0.006]* [120, 80] = [0.5, 0.5]
    self.portfolio_value = sum((initial_weights / self.data.close.values) * self.data.close.values)

    self.portfolio_return_memory = [0]
    self.asset_memory = [self.initial_amount]
    # agregar + 1
    self.actions_memory = [initial_weights / self.data.close.values]
    self.date_memory = [self.data.date.unique()[0]] 

  def reset(self):
    self.day = self.day_start
    self._reset()
    return self.state

  def save_asset_memory(self):
    date_list = self.date_memory
    portfolio_return = self.portfolio_return_memory
    df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
    return df_account_value

  def save_action_memory(self):
    # date and close price length must match actions length
    date_list = self.date_memory
    df_date = pd.DataFrame(date_list)
    df_date.columns = ['date']
    
    action_list = self.actions_memory
    df_actions = pd.DataFrame(action_list)
    df_actions.columns = self.data.tic.values
    df_actions.index = df_date.date
    return df_actions

  def render(self, mode='human'):
    return self.state

  def softmax_normalization(self, actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

  def _seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def get_sb_env(self):
    e = DummyVecEnv([lambda: self])
    obs = e.reset()
    return e, obs

  def portfolio_risk(self, weights, data_stddev, data_corr):
    aux_1 = 0
    aux_2 = 0
    for i in range(len(weights)):
        aux_1 += math.pow(weights[i], 2) * math.pow(data_stddev[i], 2)

    for i in range(len(weights)):
        for j in range(len(weights)):
            if i == j:
                continue
            aux_2 += weights[i] * weights[j] * data_stddev[i] * data_stddev[j] * data_corr.values[i][j]

    return np.sqrt(aux_1 + (2 * aux_2))

  def sharpeVE_ratio(self, weights, expected_values,
                     risk, free_risk_return):
    sr = (sum(weights * expected_values * 252) - free_risk_return) / risk
    return sr

def get_dataframe(tickers, start_date, end_date):
    df = YahooDownloader(start_date=start_date,
                         end_date=end_date,
                         ticker_list=tickers).fetch_data()
    return df

def retrieve_data_from_yahoo(assets, start_date, end_date, attribute):
  '''
  -> Example output
                  GFNORTEO.MX     BSMXB.MX    ....
  Date                              
  2019-01-02      93.142639       24.001934
  ...
  ...
  2019-12-31      93.142639       24.001934
  '''
  df = pd.DataFrame()
  for asset in assets:
    try:
      df_asset = pd.read_csv('./data/{}+[{}]_[{}].csv'.format(asset, start_date, end_date))
      df_asset.set_index('Date', inplace=True)
    except FileNotFoundError as e:
      df_asset = pdr.get_data_yahoo(asset, start=start_date, end=end_date)[attribute]
      df_asset = df_asset.to_frame(name=asset)
      df_asset.to_csv('./data/{}+[{}]_[{}].csv'.format(asset, start_date, end_date))

    df = pd.concat([df, df_asset], axis=1, sort=False)
  return df

def data_cleaning(dataframe):
    df = dataframe.dropna(axis=1, how='all') 
    df = df.interpolate(limit_direction='both') 
    return df.sort_index(axis=1), sorted(df.columns)

def data_preparation(dataframe, assets):
  data_pct_changes = dataframe.pct_change().dropna()
  data_covariances = np.cov(data_pct_changes.transpose())
  data_stddev = np.array(np.std(data_pct_changes))
  data_corr = data_pct_changes.corr()
  expected_values = get_expected_values(dataframe, data_pct_changes, assets)

  return data_stddev, data_corr, expected_values
  #return data_pct_changes, data_covariances, data_stddev, data_corr, expected_values

def get_expected_values(data, data_pct_changes, assets):
  expected_values = []

  for serie in assets:
      x, rangos = np.histogram(data_pct_changes[serie], bins=10)
      y=[]
      for i in range(len(rangos) - 1):
          y.append((rangos[i] + rangos[i + 1]) / 2)
      p = x / len(data)
      expected_value = sum(p * y) 
      expected_values.append(expected_value)

  return np.array(expected_values)

def feature_engineering(df, indicators):
    fe = FeatureEngineer(use_technical_indicator=True,
                        tech_indicator_list=indicators,
                        use_turbulence=False,
                        user_defined_feature=False)

    return fe.preprocess_data(df)

def zscore(df, tickers, properties):
  df = df.copy()
  df = df.sort_values(by=['tic','date'])

  for var in properties:
      var_df = pd.DataFrame()
      for ticker in tickers:
          temp_normalized = stats.zscore(df[df.tic == ticker][var])
          temp_normalized = pd.DataFrame(temp_normalized, columns=[var+'_zscore'])
          temp_normalized['tic'] = ticker
          temp_normalized['date'] = df[df.tic == ticker]['date'].to_list()
          var_df = var_df.append(
              temp_normalized, ignore_index=True
          )
      df = df.merge(var_df[['tic','date',var+'_zscore']],on=['tic','date'],how='left')
  df = df.sort_values(by=['date','tic'])
  return df

# Hiperparametros del PPO

se crea el agente y el entorno. 

Se pueden modificar las epocas del PPO

In [7]:
def train_and_perform(index,
                      policy,
                      episodes,
                      n_steps,
                      market_variables,
                      indicators,
                      reward_function,
                      normalization_function,
                      tickers,
                      training_start_date,
                      training_end_date,
                      trading_start_date,
                      trading_end_date,
                      trading_fee,
                      file_,
                      nn_arch='MLP',
                      window_width=1):
  df = get_dataframe(tickers, training_start_date, trading_end_date)
  df['rate open/close'] = df['open']/df['close']
  df = feature_engineering(df, indicators)

  if not (normalization_function is None):
    df = normalization_function(df, tickers, market_variables + indicators)

  train_df = data_split(df, training_start_date, training_end_date)
  print(train_df)
  # calcular vector varianzas de train_df en base a los precios de cierre
  trading_df = data_split(df, trading_start_date, trading_end_date)

  if reward_function == 'pv':
    env_kwargs = {
    'initial_amount': 1, 
    'stock_dim': len(tickers),
    'tech_indicator_list': indicators,
    'market_variables': market_variables,
    'reward_function': reward_function,
    'trading_fee': trading_fee,
    'nn_arch': nn_arch,
    'window_width': window_width,
    'day': window_width - 1
    }
  elif reward_function == 'sr':
    yf.pdr_override()

    df_adj_close = retrieve_data_from_yahoo(tickers,
                                            training_start_date,
                                            training_end_date,
                                            'Adj Close')
    df_adj_close, tickers_adj_close = data_cleaning(df_adj_close)

    data_stddev, data_corr, expected_values = data_preparation(df_adj_close, tickers_adj_close)

    free_risk_return = 0.015

    env_kwargs = {
      'initial_amount': 1, 
      'stock_dim': len(tickers), 
      'tech_indicator_list': indicators, 
      'market_variables': market_variables,
      'expected_values': expected_values,
      'reward_function': reward_function,
      'free_risk_return': free_risk_return,
      'data_stddev': data_stddev,
      'data_corr': data_corr,
      'trading_fee': trading_fee,
      'nn_arch': nn_arch,
      'window_width': window_width,
      'day': window_width - 1
    }

  training_env_gym = PortfolioAllocation(df=train_df, **env_kwargs)
  training_env, _ = training_env_gym.get_sb_env()

  trading_env_gym = PortfolioAllocation(df=trading_df, **env_kwargs)
  trading_env, _ = trading_env_gym.get_sb_env()

  ####### initialize environment hyperparameters ######

  has_continuous_action_space = True
  max_ep_len = len(train_df.date.unique())
  max_ep_trading_len = len(trading_df.date.unique())                   
  max_training_timesteps = len(train_df.date.unique()) * episodes 

  print_freq = max_ep_len * 10        # print avg reward in the interval (in num timesteps)
  log_freq = max_ep_len * 2           # log avg reward in the interval (in num timesteps)
  save_model_freq = int(1e5)          # save model frequency (in num timesteps)

  action_std = 0.6                    # starting std for action distribution (Multivariate Normal)
  action_std_decay_rate = 0.01        # linearly decay action_std (action_std = action_std - action_std_decay_rate)
  min_action_std = 0.1                # minimum action_std (stop decay after action_std <= min_action_std)
  action_std_decay_freq = int(5.5e3)  # action_std decay frequency (in num timesteps)
  
  #####################################################

  ################ PPO hyperparameters ################

  update_timestep = max_ep_len * 4      # update policy every n timesteps
  #update_timestep = int(max_ep_len / 4)
  K_epochs = 80               # update policy for K epochs in one PPO update

  eps_clip = 0.2          # clip parameter for PPO
  gamma = 0.99           # discount factor

  lr_actor = 0.0003       # learning rate for actor network
  lr_critic = 0.001       # learning rate for critic network

  random_seed = 0         # set random seed if required (0 = no random seed)

  #####################################################

  # state space dimension
  state_dim = training_env.observation_space.shape[0]

  # action space dimension
  action_dim = training_env.action_space.shape[0]

  ppo_agent = PPO(state_dim,
                  action_dim, #[0.2, 0.3, 0.5] aunque solamente tengas 2 activos
                  lr_actor,
                  lr_critic,
                  gamma,
                  K_epochs,
                  eps_clip,
                  has_continuous_action_space,
                  action_std,
                  nn_arch=nn_arch,
                  kernel_size=(len(market_variables) + len(indicators)) * window_width)
  
  # printing and logging variables
  print_running_reward = 0
  print_running_episodes = 0

  time_step = 0
  i_episode = 0

  ################ Training ################
  training_time = time.time()
  while time_step <= max_training_timesteps:
    state = training_env.reset()
    current_ep_reward = 0

    for t in range(1, max_ep_len+1):
      # select action with policy
      action = ppo_agent.select_action(state)
      state, reward, done, _ = training_env.step([action])

      reward = reward[0]

      # saving reward and is_terminals
      ppo_agent.buffer.rewards.append(reward)
      ppo_agent.buffer.is_terminals.append(done)

      time_step +=1
      current_ep_reward += reward

      # update PPO agent
      if time_step % update_timestep == 0:
        ppo_agent.update()

      # if continuous action space; then decay action std of ouput action distribution
      if has_continuous_action_space and time_step % action_std_decay_freq == 0:
        ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)

      # break; if the episode is over
      if done:
        break

    print_running_reward += current_ep_reward
    print_running_episodes += 1

    i_episode += 1
  training_time = time.time() - training_time
  print('Training')
  
  #####################################################

  ################ Testing ################

  testing_time = 0
  df_daily_return = None
  total_test_episodes = 15
  for ep in range(total_test_episodes):
    testing_time_  = time.time()
    ep_reward = 0
    state = trading_env.reset()

    for t in range(max_ep_trading_len):
      action = ppo_agent.select_action(state)
      state, reward, done, _ = trading_env.step([action])
  
      reward = reward[0]
      ep_reward += reward

      if t == max_ep_trading_len - (window_width + 2):
        tmp_dr = trading_env.env_method(method_name="save_asset_memory")[0]
        actions_memory = trading_env.env_method(method_name="save_action_memory")[0]
        if df_daily_return is None:
          df_daily_return = tmp_dr
        else:
          df_daily_return['daily_return'] = df_daily_return['daily_return'] + tmp_dr['daily_return']
      if done:
        break

    # clear buffer
    ppo_agent.buffer.clear()
    testing_time += (time.time() - testing_time_)

  print('Testing')
  print(actions_memory)
  #####################################################

  df_daily_return.loc[:,'daily_return'] /= total_test_episodes
  df_plot = deepcopy(df_daily_return)
  df_plot['date'] = pd.to_datetime(df_plot['date'])
  df_plot.set_index("date", inplace=True, drop=True)
  df_plot.index = df_plot.index.tz_localize("UTC")

  serie = pd.Series(df_plot["daily_return"], index=df_plot.index)
  print(serie)
  file_.write('training_time {}\n'.format(training_time))
  file_.write('testing_time {}\n'.format(testing_time))
  file_.write(str(pyfolio.timeseries.perf_stats(returns=serie)))
  serie.to_csv('2019-drl{}-{}-{}-{}.csv'.format(window_width, index, reward_function, normalization_function))
  #print(pyfolio.timeseries.perf_stats(returns=serie))
  pyfolio.create_full_tear_sheet(returns=serie)

# Ejecución del algoritmo

Episodios se pueden modificar

Market variables también, pero hay que revisar el DF

Indicadores tambien asi se queda

Las variables y los indicadores dan como resultado todas las combinaciones

Funcion de recompensa puede ser SR y PV

Activos

Lag

A la salida el setting actor output action_std es que hace mas chica la campana

In [8]:
policies = ['MlpPolicy'] #SE QUEDA ASI
episodes = [100] #MODIFICABLE, SE QUEDA ASI POR CONEGUNDES

market_variables = [['open', 'close', 'volume'], ['volume']] #SE PUEDE PONER EL CIERRE AJUSTADO
indicators = [['close_5_sma', 'close_10_sma', 'close_5_ema', 'close_10_ema', 'macd', 'rsi_14', 'cci', 'atr', 'boll_ub', 'boll_lb'],
              ['close_20_sma', 'boll_ub', 'boll_lb', 'rsi_14']]


reward_function = ['pv'] #SE MODIFICA DE ACUERDO A LA CORRIDA, SE PUEDE CORRER CON ['pv', 'sr']
n_steps = [None]
normalization_function = [None]

#tickers=['PLTR', 'DELL', 'GLW', 'LSPD', 'ST', 'VNT', 'SAIC', 'YALA']
#tickers=['ELAT', 'OSH', 'AMWL', 'INSP', 'CTLT', 'BIO', 'TARO', 'BSX', 'AMN', 'TEVA']
#tickers=['PDS', 'WLL', 'PBA', 'ALIN-PE', 'EURN', 'XOM', 'TPL', 'DLNG-PA', 'OVV', 'EPD']
# 2017
#tickers=['ITUB4.SA', 'BBDC4.SA', 'ABEV3.SA', 'PETR4.SA', 'VALE3.SA', 'BRFS3.SA', 'BBAS3.SA', 'ITSA4.SA', 'B3SA3.SA', 'UGPA3.SA']
# 2018
#tickers=['ITUB4.SA', 'VALE3.SA', 'BBDC4.SA', 'ABEV3.SA', 'PETR4.SA', 'B3SA3.SA', 'ITSA4.SA', 'BBAS3.SA', 'UGPA3.SA', 'BRFS3.SA']
# 2019
#tickers=['ITUB4.SA', 'VALE3.SA', 'BBDC4.SA', 'PETR4.SA', 'ABEV3.SA', 'BBAS3.SA', 'B3SA3.SA', 'ITSA4.SA', 'LREN3.SA', 'UGPA3.SA']
# SP500 - 2020
#tickers=['MSFT', 'AAPL']
# BOVESPA ETF
tickers=['ITUB4.SA','BBDC4.SA','ABEV3.SA', 'PETR4.SA', 'VALE3.SA','PETR3.SA', 'BBAS3.SA', 'ITSA4.SA', 'BRFS3.SA','UGPA3.SA','CIEL3.SA', 'JBSS3.SA','BBSE3.SA','BBDC3.SA','LREN3.SA','CCRO3.SA','RADL3.SA','EMBR3.SA','SANB11.SA','EQTL3.SA','HYPE3.SA','SBSP3.SA','GGBR4.SA','WEGE3.SA','BRKM5.SA','BRML3.SA','CPFE3.SA','CMIG4.SA','EGIE3.SA','KLBN11.SA','CSNA3.SA','CSAN3.SA','RENT3.SA','ELET3.SA','MULT3.SA','BRAP4.SA','QUAL3.SA','MRVE3.SA','ENBR3.SA','CPLE6.SA','GOAU4.SA','CYRE3.SA','USIM5.SA','MRFG3.SA','ECOR3.SA']

training_start_date='2016-01-01'
training_end_date='2017-12-31'
trading_start_date='2018-01-01'
trading_end_date='2018-12-31'

f= open("./out.txt","w+")
f.write(str(tickers) + '\n')
f.write(training_start_date+ '\n')
f.write(training_end_date+ '\n')
f.write(trading_start_date+ '\n')
f.write(trading_end_date+ '\n\n')

for i, cb in enumerate(itertools.product(policies,
                                episodes,
                                n_steps,
                                market_variables,
                                indicators,
                                reward_function,
                                normalization_function)):
  
  f.write('CB' + str(i) + '\n')
  f.write('{}\n{}\n{}\n{}\n{}\n{}\n{}\n'.format(cb[0],cb[1], cb[2], cb[3], cb[4], cb[5], cb[6]))
  train_and_perform(index='CB' + str(i),
                    policy=cb[0],
                    episodes=cb[1],
                    n_steps=None,
                    market_variables=cb[3],
                    indicators=cb[4],
                    reward_function=cb[5],
                    normalization_function=cb[6],
                    tickers=tickers,
                    training_start_date=training_start_date,
                    training_end_date=training_end_date,
                    trading_start_date=trading_start_date,
                    trading_end_date=trading_end_date,
                    trading_fee=0,
                    file_=f,
                    nn_arch='MLP',
                    window_width=5) #LAG
  f.write('\n\n\n')
f.close()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/usr/local/lib/python3.7/dist-packages/finrl/marketdata/yahoodownloader.py:69: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop("adjcp", 1)


Shape of DataFrame:  (33705, 8)
Successfully added technical indicators
           date       open       high  ...       atr    boll_ub    boll_lb
0    2016-01-04  17.730000  17.730000  ...  3.509300  14.647763  14.016737
0    2016-01-04  14.440000  14.570000  ...  3.400412  14.647763  14.016737
0    2016-01-04  10.452175  10.597057  ...  3.269908  14.548197  14.097023
0    2016-01-04  10.825763  10.973749  ...  3.199397  14.662571  13.782960
0    2016-01-04  23.850000  24.150000  ...  3.217245  14.594391  13.804041
..          ...        ...        ...  ...       ...        ...        ...
502  2017-12-29  34.330002  34.330002  ...  1.934575  21.228147  19.053770
502  2017-12-29  37.500000  37.500000  ...  1.943872  21.318277  18.827589
502  2017-12-29   9.100000   9.100000  ...  1.981404  21.291976  18.751094
502  2017-12-29  40.259998  40.259998  ...  1.963694  21.247254  18.647671
502  2017-12-29   9.273076   9.273076  ...  1.974344  21.140938  18.619447

[22635 rows x 19 columns]
[

Start date,2018-01-08
End date,2018-12-27
Total months,11
,Backtest
Annual return,6.973%
Cumulative returns,6.659%
Annual volatility,22.437%
Sharpe ratio,0.41
Calmar ratio,0.33
Stability,0.00
Max drawdown,-21.213%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,21.21,2018-02-23,2018-06-18,2018-11-01,180
1,5.40,2018-11-05,2018-11-13,2018-12-03,21
2,5.36,2018-02-01,2018-02-09,2018-02-21,15
3,3.52,2018-12-03,2018-12-20,NaT,NaN
4,1.52,2018-01-17,2018-01-23,2018-01-24,6


...

Shape of DataFrame:  (33705, 8)
Successfully added technical indicators
           date       open       high  ...    boll_ub    boll_lb      rsi_14
0    2016-01-04  17.730000  17.730000  ...  14.647762  14.016736  100.000000
0    2016-01-04  14.440000  14.570000  ...  14.647762  14.016736  100.000000
0    2016-01-04  10.452175  10.597057  ...  14.548196  14.097022   59.592518
0    2016-01-04  10.825763  10.973749  ...  14.662570  13.782960   27.367700
0    2016-01-04  23.850000  24.150000  ...  14.594390  13.804041   43.190120
..          ...        ...        ...  ...        ...        ...         ...
502  2017-12-29  34.330002  34.330002  ...  21.228144  19.053772   32.104776
502  2017-12-29  37.500000  37.500000  ...  21.318274  18.827590   36.519787
502  2017-12-29   9.100000   9.100000  ...  21.291975  18.751096   46.509124
502  2017-12-29  40.259998  40.259998  ...  21.247252  18.647674   43.298904
502  2017-12-29   9.273076   9.273076  ...  21.140936  18.619450   47.636105

[22

Start date,2018-01-08
End date,2018-12-27
Total months,11
,Backtest
Annual return,5.727%
Cumulative returns,5.47%
Annual volatility,21.846%
Sharpe ratio,0.36
Calmar ratio,0.29
Stability,0.00
Max drawdown,-19.626%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,19.63,2018-02-23,2018-06-18,2018-11-05,182
1,5.73,2018-01-26,2018-02-09,2018-02-23,21
2,5.56,2018-11-05,2018-11-13,2018-11-29,19
3,3.71,2018-12-03,2018-12-26,NaT,NaN
4,1.49,2018-01-17,2018-01-23,2018-01-24,6


/usr/local/lib/python3.7/dist-packages/pyfolio/timeseries.py:1230: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  period = returns_dupe.loc[start:end]


Stress Events,mean,min,max
New Normal,0.03%,-4.58%,5.25%
